https://github.com/gilbutITbook/080289/blob/main/chap07/python_7%EC%9E%A5.ipynb
를 기준으로 실습 진행, 책 내용을 추가로 정리하였다.

In [2]:
import torch
import torchtext
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import time

In [4]:
TEXT = torchtext.legacy.data.Field(lower = True, fix_length = 200, batch_first = False)
# lower : 대문자를 모두 소문자로 변경해준다. 기본값이 False.
# fix_length : 고정된 길이로 맞춘다. (패딩 작업 실시)
# batch_first : 신경망 입력의 첫 차원 값이 배치 크기가 되게 한다. 
  # 기본적으로는 시퀀스 길이 , 배치 크기, 은닉층 뉴런 개수로 들어간다.
  # batch_first true일 경우 배치 크기, 시퀀스 길이, 은닉층 뉴런 개수

LABEL = torchtext.legacy.data.Field(sequential = False)

예제로는 IMDB를 다시 써본다.

In [5]:
from torchtext.legacy import datasets
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

downloading aclImdb_v1.tar.gz


100%|██████████| 84.1M/84.1M [00:37<00:00, 2.23MB/s]


In [6]:
print(vars(train_data.examples[0]))

{'text': ['bromwell', 'high', 'is', 'a', 'cartoon', 'comedy.', 'it', 'ran', 'at', 'the', 'same', 'time', 'as', 'some', 'other', 'programs', 'about', 'school', 'life,', 'such', 'as', '"teachers".', 'my', '35', 'years', 'in', 'the', 'teaching', 'profession', 'lead', 'me', 'to', 'believe', 'that', 'bromwell', "high's", 'satire', 'is', 'much', 'closer', 'to', 'reality', 'than', 'is', '"teachers".', 'the', 'scramble', 'to', 'survive', 'financially,', 'the', 'insightful', 'students', 'who', 'can', 'see', 'right', 'through', 'their', 'pathetic', "teachers'", 'pomp,', 'the', 'pettiness', 'of', 'the', 'whole', 'situation,', 'all', 'remind', 'me', 'of', 'the', 'schools', 'i', 'knew', 'and', 'their', 'students.', 'when', 'i', 'saw', 'the', 'episode', 'in', 'which', 'a', 'student', 'repeatedly', 'tried', 'to', 'burn', 'down', 'the', 'school,', 'i', 'immediately', 'recalled', '.........', 'at', '..........', 'high.', 'a', 'classic', 'line:', 'inspector:', "i'm", 'here', 'to', 'sack', 'one', 'of', '

각 데이터는 텍스트와, label(긍정/부정)으로 구성되어 있다.

In [7]:
# 텍스트 전처리
import string

for example in train_data.examples:
    text = [x.lower() for x in vars(example)['text']] 
    text = [x.replace("<br","") for x in text] 
    text = [''.join(c for c in s if c not in string.punctuation) for s in text] 
    text = [s for s in text if s] 
    vars(example)['text'] = text
    
for example in test_data.examples:
    text = [x.lower() for x in vars(example)['text']]
    text = [x.replace("<br","") for x in text]
    text = [''.join(c for c in s if c not in string.punctuation) for s in text]
    text = [s for s in text if s]
    vars(example)['text'] = text

In [8]:
import random
train_data, valid_data = train_data.split(random_state = random.seed(0), split_ratio=0.8)

In [11]:
# 단어 집합 만들기
TEXT.build_vocab(train_data, max_size = 10000, min_freq = 10, vectors = None)
# max_size : 단어 집합에 포함되는 어휘 수
# min_freq : 최소 등장 횟수컷
# vectors : 단어를 랜덤한 숫자로 변형하는 벡터화(임베딩)
LABEL.build_vocab(train_data)

전처리 과정이 완료되어, 이제 데이터를 메모리로 불러온다.

In [12]:
BATCH_SIZE = 64
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

embeding_dim = 100  # 은닉층의 유닛 개수를 지정한다.
hidden_size = 300

# data_loader와 유사한 느낌으로 쓰이는 BucketIterator.
# 배치 단위로 데이터를 차례대로 꺼내, 메모리로 가져온다.
train_iterator, valid_iterator, test_iterator = torchtext.legacy.data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

## 셀에 대한 처리

In [13]:
class RNNCell_Encoder(nn.Module):
    def __init__(self, input_dim, hidden_size):
        super(RNNCell_Encoder, self).__init__()
        self.rnn = nn.RNNCell(input_dim, hidden_size)

    def forward(self, inputs):   # input : 입력 시퀀스(시퀀스 길이, 배치, 임베딩)
        bz = inputs.shape[1] # 배치 하나를 가져온다.
        ht = torch.zeros((bz, hidden_size)).to(device) # 뉴런 크기의 초기화.

        for word in inputs:
            ht = self.rnn(word, ht) # 현재의 입력 상태, 이전 상태
        return ht

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.em = nn.Embedding(len(TEXT.vocab.stoi), embeding_dim) # text_embedding 처리.
        self.rnn = RNNCell_Encoder(embeding_dim, hidden_size)
        self.fc1 = nn.Linear(hidden_size, 256)
        self.fc2 = nn.Linear(256, 3)

    def forward(self, x):
        x = self.em(x)
        x = self.rnn(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [14]:
model = Net()
model.to(device)

loss_fn = nn.CrossEntropyLoss() # 다중 분류에 사용된다.
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [17]:
def training(epoch, model, trainloader, validloader):
    correct = 0
    total = 0
    running_loss = 0

    model.train()
    for b in trainloader:
        x, y = b.text, b.label  # train_loader에서 text, label 구현.
        x, y = x.to(device), y.to(device) # cpu냐, gpu냐 판단하는 과정.
        y_pred = model(x)
        loss = loss_fn(y_pred, y) # crossentropyloss 손실 함수 이용, 오차 계산.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        with torch.no_grad():
            y_pred = torch.argmax(y_pred, dim=1)
            correct += (y_pred == y).sum().item()
            total += y.size(0)
            running_loss += loss.item()
    epoch_loss = running_loss / len(trainloader.dataset) # 누적된 오차, 전체 데이터셋으로 나눠 오차 계산.
    epoch_acc = correct / total

    valid_correct = 0
    valid_total = 0
    valid_running_loss = 0

    model.eval()
    with torch.no_grad():
        for b in validloader:
            x, y = b.text, b.label
            x, y = x.to(device), y.to(device)
            y_pred = model(x)
            loss = loss_fn(y_pred, y)
            y_pred = torch.argmax(y_pred, dim=1)
            valid_correct += (y_pred == y).sum().item()  # 맞은 개수
            valid_total += y.size(0)
            valid_running_loss += loss.item()

    epoch_valid_loss = valid_running_loss / len(validloader.dataset)
    epoch_valid_acc = valid_correct / valid_total

    print('epoch: ', epoch,
          'loss： ', round(epoch_loss, 3),
          'accuracy:', round(epoch_acc, 3),
          'valid_loss： ', round(epoch_valid_loss, 3),
          'valid_accuracy:', round(epoch_valid_acc, 3)
          )  # 훈련 진행 단계마다, 오차 출력하게 한다.
    return epoch_loss, epoch_acc, epoch_valid_loss, epoch_valid_acc

In [18]:
epochs = 5
train_loss = []
train_acc = []
valid_loss = []
valid_acc = []

for epoch in range(epochs):
    epoch_loss, epoch_acc, epoch_valid_loss, epoch_valid_acc = training(epoch, model, train_iterator, valid_iterator)
    train_loss.append(epoch_loss)
    train_acc.append(epoch_acc)
    valid_loss.append(epoch_valid_loss)
    valid_acc.append(epoch_valid_acc)

epoch:  0 loss：  0.011 accuracy: 0.498 valid_loss：  0.011 valid_accuracy: 0.506
epoch:  1 loss：  0.011 accuracy: 0.501 valid_loss：  0.011 valid_accuracy: 0.495
epoch:  2 loss：  0.011 accuracy: 0.512 valid_loss：  0.011 valid_accuracy: 0.496
epoch:  3 loss：  0.011 accuracy: 0.521 valid_loss：  0.011 valid_accuracy: 0.491
epoch:  4 loss：  0.011 accuracy: 0.526 valid_loss：  0.011 valid_accuracy: 0.513


NameError: name 'start' is not defined

## RNN 계층 학습

In [19]:
vocab_size = len(TEXT.vocab)
n_classes = 2  

In [20]:
class BasicRNN(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p = 0.2):
        super(BasicRNN, self).__init__()
        self.n_layers = n_layers   # RNN 계층 개수.
        self.embed = nn.Embedding(n_vocab, embed_dim)  # Word, Embedding 적용
        self.hidden_dim = hidden_dim
        self.dropout = nn.Dropout(dropout_p)  # Dropout.
        self.rnn = nn.RNN(embed_dim, self.hidden_dim, num_layers = self.n_layers, batch_first = True) 
        # 특성 개수, 은닉층 개수, 계층 개수, 
        self.out = nn.Linear(self.hidden_dim, n_classes) # 최종 출력.

    def forward(self, x):
        x = self.embed(x) 
        h_0 = self._init_state(batch_size = x.size(0)) # 최초 은닉층의 초기화
        x, _ = self.rnn(x, h_0)  # parameter로 입력과, 이전 층의 은닉을 받는다.
        h_t = x[:, -1, :]  # 마지막 은닉.
        self.dropout(h_t)
        logit = torch.sigmoid(self.out(h_t))
        return logit

    def _init_state(self, batch_size = 1):
        weight = next(self.parameters()).data 
        return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()

In [21]:
model = BasicRNN(n_layers = 1, hidden_dim = 256, n_vocab = vocab_size, embed_dim = 128, n_classes = n_classes, dropout_p = 0.5)
model.to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [22]:
def train(model, optimizer, train_iter):
    model.train()
    for b, batch in enumerate(train_iter):
        x, y = batch.text.to(device), batch.label.to(device)
        y.data.sub_(1)  # label 값이 0 or 1이 되버린다.
        optimizer.zero_grad()

        logit = model(x)
        loss = F.cross_entropy(logit, y)
        loss.backward()
        optimizer.step()

        if b % 50 == 0:
            print("Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(e,
                                                                           b * len(x),
                                                                           len(train_iter.dataset),
                                                                           100. * b / len(train_iter),
                                                                           loss.item()))

In [23]:
def evaluate(model, val_iter):
    model.eval()
    corrects, total, total_loss = 0, 0, 0

    for batch in val_iter:
        x, y = batch.text.to(device), batch.label.to(device)
        y.data.sub_(1) 
        logit = model(x)
        loss = F.cross_entropy(logit, y, reduction = "sum")
        total += y.size(0)
        total_loss += loss.item()
        corrects += (logit.max(1)[1].view(y.size()).data == y.data).sum()
        
    avg_loss = total_loss / len(val_iter.dataset)
    avg_accuracy = corrects / total
    return avg_loss, avg_accuracy

In [24]:
BATCH_SIZE = 100
LR = 0.001
EPOCHS = 5
for e in range(1, EPOCHS + 1):
    train(model, optimizer, train_iterator)
    val_loss, val_accuracy = evaluate(model, valid_iterator)
    print("[EPOCH: %d], Validation Loss: %5.2f | Validation Accuracy: %5.2f" % (e, val_loss, val_accuracy))

ValueError: Expected input batch_size (200) to match target batch_size (64).

RNN의 결정적인 단점은, 기울기가 1보다 작은 값이 계속 곱해진다. 기울기가 사라지는 기울기 소멸 문제 발생.
-> LSTM,  GRU와 같은 확장된 RNN 방식을 사용한다.